# OpenEO: Sentinel 1, $\sigma_0$

Connection to VITO backend for cloud computing

In [ ]:
import openeo
import geopandas as gpd
from pathlib import Path
import numpy as np
import pickle
import xarray as xr
import rioxarray
import hvplot.xarray
import hvplot.dask
import matplotlib.pyplot as plt
import os
import time 
connection = openeo.connect("openeo.vito.be").authenticate_oidc()
pad = Path(os.getcwd())
if pad.name != "Python":
    pad_correct = Path("../../Python")
    os.chdir(pad_correct)
#set all of the parameters below to True to execute cloud computing and downloading    
overwrite = False
read = True
job_exec = False
download = True
write_out = True

Load in the Shapefile of the Zwalm in EPSG:4326

In [ ]:
if not os.path.exists('data/Zwalm_shape/zwalm_shapefile_emma.shp'):
    %run "preprocessing_files/shapefile_conversion.py"   
    print('Preprocessing script of Zwalm shapefile has run')

In [ ]:
shape_zwalm = gpd.read_file('data/Zwalm_shape/zwalm_shapefile_emma.shp')
shape_zwalm.plot()
extent = shape_zwalm.total_bounds
print(extent)

In [ ]:
#connection.list_collections()

In [ ]:
#connection.describe_collection('SENTINEL1_GRD')
connection.describe_collection('S1_GRD_SIGMA0_ASCENDING')

Splitting the temporal extent in to 1 year at a time

In [ ]:
temporal_extent = ["2015-06-07", "2022-11-05"]
list_temp_extent = []
list_temp_extent.append([temporal_extent[0],"2015-12-31"])
years = np.arange(2016,2023)
for year in np.arange(2016,2023):
    if year == 2022:
        #print([str(year)+"-01-01",temporal_extent[1]])
        list_temp_extent.append([str(year)+"-01-01",temporal_extent[1]])
    else:
        #print([str(year)+"-01-01",str(year)+ "-12-31"])
        list_temp_extent.append([str(year)+"-01-01",str(year)+ "-12-31"])
print(list_temp_extent)

## Ascending orbit(s)

In [ ]:
collection = 'S1_GRD_SIGMA0_ASCENDING' #Ground Range Detected
spatial_extent = {'west':extent[0],'east':extent[2],'south':extent[1],'north':extent[3]}
bands = ["VV","VH","angle"] #enkel in deze geïnteresseerd (add VH: 24/02/2022)
properties = {
    'sat:orbit_state': lambda od: od == "ASCENDING"
}

In [ ]:
years = np.arange(2015,2023)
job_title_list = []
job_id_list = []
if job_exec:
    for i, temporal_extent in enumerate(list_temp_extent):
        s1a = connection.load_collection(
            collection_id = collection,
            spatial_extent= spatial_extent,
            temporal_extent = temporal_extent,
            bands = bands
        )
        s1a = s1a.mask_polygon(shape_zwalm['geometry'].values[0])
        job_title = "s1_a_terrascope-" +  str(years[i])
        job_title_list.append(job_title)
        job_s1a = s1a.create_job(title = job_title, out_format= 'NetCDF')
        job_s1a_id = job_s1a.job_id
        if job_s1a_id:
            print("Batch job created with id: ",job_s1a_id)
            job_s1a.start_job()
            job_id_list.append(job_s1a_id)
        else:
            print("Error! Job ID is None")

In [ ]:
if not os.path.exists('data/s0_OpenEO/ascending'):
    os.makedirs('data/s0_OpenEO/ascending')
if download:
    for i,job_id in enumerate(job_id_list):
        job_connection = connection.job(job_id)
        results = job_connection.get_results()
        name_netcdf = job_title_list[i] + '.nc'
        filepath = "data/s0_OpenEO/ascending/" + name_netcdf
        print(filepath)
        while job_connection.status() != 'finished':
            time.sleep(30)
            if job_connection.status() == 'error':
                raise ChildProcessError(job_id + 'has encountered an error, check why batch job failed')
            if job_connection.status() == 'canceled':
                        raise ChildProcessError(job_id + 'has been canceled')
        results.download_file(filepath)

Read in the downloaded data! 
- scale and offset from https://docs.terrascope.be/DataProducts/Sentinel-1/references/VITO_S1_sigma0_GRD.pdf 

In [ ]:
s1_xr_asc = xr.open_mfdataset('data/s0_OpenEO/ascending/*.nc', decode_coords="all") #automatically chuncked!
#s1_xr_asc['VV_db'] = 10 * np.log10(s1_xr_asc['VV'])
scale = 0.0005
offset = 29
s1_xr_asc['angle']  = s1_xr_asc['angle']*scale + offset   
s1_xr_asc

Now only select the values for which sufficient data is present. First check amount of data in a full image

In [ ]:
s1_xr_asc_plot = s1_xr_asc.copy()
s1_xr_asc_plot['VV_db'] = 10*np.log10(s1_xr_asc_plot['VV'])
s1_xr_asc_plot['VV_db'].isel(t=0).plot() #example of a full image

In [ ]:
s1_xr_asc['angle'].isel(t=0).plot()

VH

In [ ]:
xr_shape = s1_xr_asc['VH'].shape
print('Shape of the ascending orbits: ' + str(xr_shape))
nr_pixels = xr_shape[1]*xr_shape[2]
print('Number of pixels per timestamp: ' + str(nr_pixels))
nancount = np.sum(np.isnan(s1_xr_asc['VV'].isel(t=0))).values
print('Number of nan-pixels for a full image: ' + str(nancount))
nan_cutoff = nancount/nr_pixels
print('Percentage of nan-pixels in a full image: ' + str(nan_cutoff*100) + '%')
#add 5% as safety margin to cutoff
nan_cutoff = nan_cutoff + 0.05

VV

In [ ]:
xr_shape = s1_xr_asc['VV'].shape
print('Shape of the ascending orbits: ' + str(xr_shape))
nr_pixels = xr_shape[1]*xr_shape[2]
print('Number of pixels per timestamp: ' + str(nr_pixels))
nancount = np.sum(np.isnan(s1_xr_asc['VV'].isel(t=0))).values
print('Number of nan-pixels for a full image: ' + str(nancount))
nan_cutoff = nancount/nr_pixels
print('Percentage of nan-pixels in a full image: ' + str(nan_cutoff*100) + '%')
#add 5% as safety margin to cutoff
nan_cutoff = nan_cutoff + 0.05

Since the goal is to select the orbits that have a full view of the Zwalm, filtering on VV and VH is applied. If there are differences, only frames will be chosen where VV and VH are both in full image

In [ ]:
nr_timestemps_ascending = xr_shape[0]
bool_full_image = []
for i in range(nr_timestemps_ascending):
    VV_ds = s1_xr_asc['VV'].isel(t=i)
    temp_nancount = np.sum(np.isnan(VV_ds)).values
    nan_frac = temp_nancount/nr_pixels
    if nan_frac > nan_cutoff:
        bool_full_image.append(0)
    else:
        bool_full_image.append(1)

Repeat for VH

In [ ]:
bool_full_image_VH = []
for i in range(nr_timestemps_ascending):
    VV_ds = s1_xr_asc['VH'].isel(t=i)
    temp_nancount = np.sum(np.isnan(VV_ds)).values
    nan_frac = temp_nancount/nr_pixels
    if nan_frac > nan_cutoff:
        bool_full_image_VH.append(0)
    else:
        bool_full_image_VH.append(1)

In [ ]:
if not bool_full_image_VH == bool_full_image:
    pos_full_VH = np.where(bool_full_image_VH)[0].tolist()
    bool_full_image_np = np.array(bool_full_image)
    bool_full_image_np[pos_full_VH] = 0
    bool_full_image = bool_full_image_np.tolist()

In [ ]:
pos_full = np.where(bool_full_image)[0].tolist()
print(pos_full)

In [ ]:
#add orbit direction
da = xr.DataArray(
    data = np.repeat('ascending',nr_timestemps_ascending),
    dims = ['t'],
    coords = dict(t = s1_xr_asc['t'].values)
)
s1_xr_asc['Orbitdirection'] = da

In [ ]:
s1_xr_asc_full = s1_xr_asc.isel(t = pos_full)
s1_xr_asc_full

In [ ]:
temp = 10*np.log10(s1_xr_asc_full['VV'])
temp.hvplot.image('x','y', geo = True, crs = 32631, tiles = 'OSM', cmap = 'bwr', width = 400, rasterize = True)

## Descending orbit(s)

In [ ]:
collection = "S1_GRD_SIGMA0_DESCENDING"
years = np.arange(2015,2023)
job_title_list_d = []
job_id_list_d = []
if job_exec:
    for i, temporal_extent in enumerate(list_temp_extent):
        s1d = connection.load_collection(
            collection_id = collection,
            spatial_extent= spatial_extent,
            temporal_extent = temporal_extent,
            bands = bands
        )
        s1d = s1d.mask_polygon(shape_zwalm['geometry'].values[0])
        job_title = "s1_d_terrascope-" +  str(years[i])
        job_title_list_d.append(job_title)
        job_s1d = s1d.create_job(title = job_title, out_format= 'NetCDF')
        job_s1d_id = job_s1d.job_id
        if job_s1d_id:
            print("Batch job created with id: ",job_s1d_id)
            job_s1d.start_job()
            job_id_list_d.append(job_s1d_id)
        else:
            print("Error! Job ID is None")

In [ ]:
if overwrite:
    with open('data/S0_OpenEO/s1_d_job_id_list.pickle', 'wb') as handle:
        pickle.dump(job_id_list_d, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('data/S0_OpenEO/s1_d_job_title_list.pickle', 'wb') as handle:
        pickle.dump(job_title_list_d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
if read:
    job_id_list_d = pickle.load(open('data/S0_OpenEO/s1_d_job_id_list.pickle', "rb"))
    job_title_list_d = pickle.load(open('data/S0_OpenEO/s1_d_job_title_list.pickle', "rb"))

In [ ]:
if not os.path.exists('data/s0_OpenEO/descending'):
    os.makedirs('data/s0_OpenEO/descending')
if download:
    for i,job_id in enumerate(job_id_list_d):
        job_connection = connection.job(job_id)
        results = job_connection.get_results()
        name_netcdf = job_title_list_d[i] + '.nc'
        filepath = "data/s0_OpenEO/descending/" + name_netcdf
        print(filepath)
        while job_connection.status() != 'finished':
            time.sleep(30)
            if job_connection.status() == 'error':
                raise ChildProcessError(job_id + 'has encountered an error, check why batch job failed')
            if job_connection.status() == 'canceled':
                        raise ChildProcessError(job_id + 'has been canceled')
        results.download_file(filepath)

Analogous processing as for ascending

In [ ]:
s1_xr_desc = xr.open_mfdataset('data/s0_OpenEO/descending/*.nc', decode_coords="all") #automatically chuncked!
# s1_xr_desc['VV_db'] = 10 * np.log10(s1_xr_desc['VV'])
s1_xr_desc['angle']  = s1_xr_desc['angle']*scale + offset   
display(s1_xr_desc)
xr_shape_desc = s1_xr_desc['VV'].shape
#only full images selected
nr_timestemps_descending = xr_shape_desc[0]
#VV
bool_full_image = []
for i in range(nr_timestemps_descending):
    VV_ds = s1_xr_desc['VV'].isel(t=i)
    temp_nancount = np.sum(np.isnan(VV_ds)).values
    nan_frac = temp_nancount/nr_pixels
    if nan_frac > nan_cutoff:
        bool_full_image.append(0)
    else:
        bool_full_image.append(1)
#VH
bool_full_image_VH = []
for i in range(nr_timestemps_ascending):
    VV_ds = s1_xr_asc['VH'].isel(t=i)
    temp_nancount = np.sum(np.isnan(VV_ds)).values
    nan_frac = temp_nancount/nr_pixels
    if nan_frac > nan_cutoff:
        bool_full_image_VH.append(0)
    else:
        bool_full_image_VH.append(1)
if not bool_full_image_VH == bool_full_image:
    pos_full_VH = np.where(bool_full_image_VH)[0].tolist()
    bool_full_image_np = np.array(bool_full_image)
    bool_full_image_np[pos_full_VH] = 0
    bool_full_image = bool_full_image_np.tolist()
pos_full = np.where(bool_full_image)[0].tolist()
#add orbit direction
da = xr.DataArray(
    data = np.repeat('descending',nr_timestemps_descending),
    dims = ['t'],
    coords = dict(t = s1_xr_desc['t'].values)
)
s1_xr_desc['Orbitdirection'] = da

s1_xr_desc_full = s1_xr_desc.isel(t= pos_full)
display(s1_xr_desc_full)

In [ ]:
temp = 10*np.log10(s1_xr_desc_full['VV'])
temp.hvplot.image('x','y', geo = True, crs = 32631, tiles = 'OSM', cmap = 'bwr', width = 400, rasterize = True)

## Combining orbits

In [ ]:
s1_xr_asc_full.attrs

In [ ]:
#s1_xr_full = xr.combine_by_coords([s1_xr_asc_full,s1_xr_desc_full], coords = ['x','y'])
s1_xr_full = xr.merge([s1_xr_asc_full, s1_xr_desc_full])

In [ ]:
display(s1_xr_full)

In [ ]:
temp = 10 * np.log10(s1_xr_full['VV'])
temp.hvplot.image('x','y', geo = True, crs = 32631, tiles = 'OSM', cmap = 'bwr', frame_width = 400, rasterize = True)

In [ ]:
if write_out:
    s1_xr_full.to_netcdf('data/S0_OpenEO/S0_zwalm.nc')